In [66]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

In [67]:
dataset = "D:\ML_DL\Examples_work__implementations\Video_classification_sports\Sports-Type-Classifier\data"

# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = ["weight_lifting", "tennis", "football"]

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

imagePaths = paths.list_images(dataset) 

data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
        # if the label of the current image is not part of of the labels
        # are interested in, then ignore the image
        if label not in LABELS:
            continue
            
        # load the image, convert it to RGB channel ordering, and resize
        # it to be a fixed 224x224 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)

[INFO] loading images...


In [68]:
print(len(data))
print(len(labels))

2070
2070


In [69]:
# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [70]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [71]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
for layer in baseModel.layers:
    layer.trainable = False

In [72]:
epochs = 30;

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
        x=trainAug.flow(trainX, trainY, batch_size=32),
        steps_per_epoch=len(trainX) // 32,
        validation_data=valAug.flow(testX, testY),
        validation_steps=len(testX) // 32,
        epochs=epochs)

[INFO] compiling model...
[INFO] training head...


C:\Users\Image903\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
48/48 [==============================] - 18s 283ms/step - loss: 1.2195 - accuracy: 0.4901 - val_loss: 0.7129 - val_accuracy: 0.7012
Epoch 2/30
48/48 [==============================] - 13s 260ms/step - loss: 0.8264 - accuracy: 0.6480 - val_loss: 0.5177 - val_accuracy: 0.8047
Epoch 3/30
48/48 [==============================] - 13s 263ms/step - loss: 0.6793 - accuracy: 0.7171 - val_loss: 0.4336 - val_accuracy: 0.8438
Epoch 4/30
48/48 [==============================] - 13s 264ms/step - loss: 0.5513 - accuracy: 0.7711 - val_loss: 0.3807 - val_accuracy: 0.8672
Epoch 5/30
48/48 [==============================] - 13s 262ms/step - loss: 0.5216 - accuracy: 0.7974 - val_loss: 0.3482 - val_accuracy: 0.8828
Epoch 6/30
48/48 [==============================] - 13s 263ms/step - loss: 0.4849 - accuracy: 0.8184 - val_loss: 0.3227 - val_accuracy: 0.8906
Epoch 7/30
48/48 [==============================] - 13s 260ms/step - loss: 0.4373 - accuracy: 0.8296 - val_loss: 0.3055 - val_accuracy: 0.9004

In [77]:
print("[INFO] evaluating network...")

predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))


[INFO] evaluating network...
17/17 [==============================] - 2s 90ms/step
                precision    recall  f1-score   support

      football       0.92      0.93      0.92       196
        tennis       0.91      0.90      0.91       179
weight_lifting       0.93      0.94      0.93       143

      accuracy                           0.92       518
     macro avg       0.92      0.92      0.92       518
  weighted avg       0.92      0.92      0.92       518



In [86]:
# plot the training loss and accuracy
N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("fig_train_loss.png", dpi=600)

AttributeError: 'list' object has no attribute 'use'

In [74]:
# serialize the model to disk
print("[INFO] serializing network...")
model.save('sports_model.h5')

[INFO] serializing network...


In [75]:
# serialize the label binarizer to disk
f = open("label_bin.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()